In [50]:
import pandas as pd
import numpy as np

In [58]:
df = pd.concat(
    [
        pd.read_hdf("../hdf/test_date.hdf"),
        pd.read_hdf("../hdf/test_categorical.hdf"),
        pd.read_hdf("../hdf/test_numeric.hdf")
    ],
    axis=1)

In [52]:
response = pd.read_hdf("../hdf/train_response.hdf")

In [59]:
X = pd.DataFrame(index=df.index, columns=[])
X["L0"] = pd.DataFrame(df.loc[:, [c for c in df.columns if "L0" in c]].T.min())
X["L1"] = pd.DataFrame(df.loc[:, [c for c in df.columns if "L1" in c]].T.min())
X["L2"] = pd.DataFrame(df.loc[:, [c for c in df.columns if "L2" in c]].T.min())
X["L3"] = pd.DataFrame(df.loc[:, [c for c in df.columns if "L3" in c]].T.min())

In [60]:
X.where(np.isnan(X), 1.0, inplace=True)
X.where(~np.isnan(X), 0.0, inplace=True)

In [61]:
X["mask"] = X["L0"] * 1000 + X["L1"] * 100 + X["L2"] * 10 + X["L3"]

In [62]:
X.drop([c for c in X.columns if c is not "mask"], axis=1, inplace=True)

In [63]:
X.to_hdf("../hdf/test_mask.hdf", "df", mode="w")

In [39]:
X = pd.concat([X, response], axis=1)

In [41]:
X[X["Response"] == 1.0]["mask"].value_counts()

1001.0    4066
111.0     1724
1011.0     836
101.0      195
1.0         30
1101.0      10
1111.0      10
11.0         4
0.0          2
1100.0       1
1010.0       1
Name: mask, dtype: int64

In [44]:
X[X["Response"] == 0.0]["mask"].value_counts()

1001.0    796883
111.0     240685
1011.0    110781
101.0      21202
1.0         2499
1111.0      2183
1101.0      1258
11.0         792
0.0          580
110.0          3
100.0          2
Name: mask, dtype: int64

In [47]:
df = pd.DataFrame(index=X["mask"].value_counts().index, columns=[])
df["OK"] = X[X["Response"] == 1.0]["mask"].value_counts()
df["NG"] = X[X["Response"] == 0.0]["mask"].value_counts()

In [49]:
df["OK"] / df.T.sum()

1001.0    0.005076
111.0     0.007112
1011.0    0.007490
101.0     0.009113
1.0       0.011862
1111.0    0.004560
1101.0    0.007886
11.0      0.005025
0.0       0.003436
110.0          NaN
100.0          NaN
1100.0    1.000000
1010.0    1.000000
dtype: float64